# Exploitation des déséquilibres Polymarket sur BTC

Ce carnet assemble la chaîne complète : chargement des données, calibration d'un modèle de cotes FOMO, estimation de la probabilité réelle de clôture d'une bougie et backtests des stratégies 2 % capital et 4 % de parts.


## Plan du carnet

1. Chargement et préparation des données (1s et 1m)
2. Simulation de cotes FOMO et visualisations
3. Entraînement du modèle de cotes vs marché
4. Entraînement du modèle de probabilité réelle
5. Génération des cotes synthétiques historiques et backtests


### Préparation des dépendances

**Objectif** : charger toutes les fonctions utilitaires définies dans `notebooks/code` nécessaires au pipeline complet.

**Sorties attendues** : aucune sortie directe. Les imports réussis garantissent que chaque module est disponible pour les cellules suivantes. En cas d'erreur, vérifier que les fichiers sont bien présents et que les dépendances Python sont installées.


In [ ]:
from pathlib import Path

import pandas as pd

from btc_code.pipeline import (
    load_all_data,
    prepare_ohlc_features,
    enrich_polymarket_with_features,
    prepare_timeframe_tables,
    build_regression_dataset,
    build_classification_dataset,
    prepare_minute_history,
    select_recent_rows,
    estimate_average_spreads,
)
from btc_code.model_training import (
    train_odds_regressors,
    train_outcome_classifiers,
    predict_regressions,
    predict_classifications,
)
from btc_code.backtest import BacktestParams, run_backtest
from btc_code.visualization import (
    plot_odds_comparison,
    plot_equity_curves,
    plot_calibration_curve,
)
from btc_code.persistence import (
    export_regression_artifacts,
    export_classification_artifacts,
)
from btc_code.pricing import probabilities_to_prices



### Chargement et enrichissement des données

**Objectif** : lire les flux Polymarket (1 seconde) et les bougies BTC (1 minute), construire les features techniques, préparer les tables par horizon (m15, h1, daily) et stocker les structures intermédiaires utiles aux modèles.

**Sorties attendues** : affichage des premières lignes de `polymarket_raw` et `ohlc_raw` pour valider le chargement. Des dictionnaires `regression_data`, `classification_data` ainsi que `minute_history` sont créés en mémoire pour une réutilisation par les cellules suivantes. Si certains champs sont `NaN`, vérifier la cohérence des horodatages ou ajuster `select_recent_rows`.


In [ ]:
polymarket_raw, ohlc_raw = load_all_data()
polymarket_subset = select_recent_rows(polymarket_raw, 500_000)
ohlc_features = prepare_ohlc_features(ohlc_raw)
per_second_base = enrich_polymarket_with_features(polymarket_subset, ohlc_features)

timeframes = ["m15", "h1", "daily"]
per_second_full = per_second_base.copy()
for tf in timeframes:
    per_second_full = prepare_timeframe_tables(per_second_full, tf)

average_spreads = estimate_average_spreads(per_second_full, timeframes)

timeframe_tables = {}
regression_data = {}
classification_data = {}
for tf in timeframes:
    tf_df = prepare_timeframe_tables(per_second_base.copy(), tf)
    timeframe_tables[tf] = tf_df
    reg_dataset, reg_features, reg_targets = build_regression_dataset(tf_df, tf)
    regression_data[tf] = {
        "dataset": reg_dataset,
        "features": reg_features,
        "targets": reg_targets,
    }
    cls_dataset, cls_features, cls_target = build_classification_dataset(tf_df, tf)
    classification_data[tf] = {
        "dataset": cls_dataset,
        "features": cls_features,
        "target": cls_target,
    }

minute_history = {tf: prepare_minute_history(ohlc_features, tf) for tf in timeframes}

polymarket_raw.head(), ohlc_raw.head()


## Sélection des contrats pour la comparaison marché / modèle

**Objectif** : choisir les identifiants de contrats (m15, h1, daily) qui seront utilisés dans les graphiques de comparaison des cotes. Cela permet de naviguer facilement d’un pari à l’autre (par exemple trois contrats m15 consécutifs, deux contrats h1, un contrat daily couvrant une journée complète).

**Sorties attendues** : un dictionnaire `selected_contracts` que l’on peut modifier à la main : changer l’ordre, le nombre ou les identifiants affichés selon les périodes que l’on souhaite analyser.

In [ ]:
daily_contracts = list(timeframe_tables["daily"]["daily_contract_id"].dropna().unique())
selected_contracts = {
    "m15": list(timeframe_tables["m15"]["m15_contract_id"].dropna().unique()[:3]),
    "h1": list(timeframe_tables["h1"]["h1_contract_id"].dropna().unique()[:2]),
    "daily": daily_contracts[:1],
}

selected_contracts


In [ ]:
# Astuce : ajuster manuellement les identifiants dans `selected_contracts`
# pour naviguer d’un pari à l’autre (contrats m15, h1, daily).


#### Visualisation des cotes marché vs modèle ML

**Objectif** : tracer, pour chaque horizon et pour les contrats sélectionnés, la cote implicite du marché et celle prédite par le modèle de régression. Les segments sont tracés contrat par contrat pour éviter les liaisons artificielles entre deux paris distincts.

**Sorties attendues** : trois graphiques (m15, h1, daily) dont le titre affiche « Comparaison des cotes <timeframe> » et la date du jour au format « 12 novembre ». L’axe Y est libellé « Probabilité de clôture UP » et chaque série (marché, modèle) dispose d’une légende explicite.


## Modèle de cotes (régression)

**Objectif** : entraîner, pour chaque horizon, un modèle de régression qui prédit le mid price Up/Down ainsi que la probabilité implicite du marché à partir des features techniques.

**Sorties attendues** : un dictionnaire `regression_artifacts` contenant les pipelines scikit-learn et un tableau `regression_metrics` (MAE/RMSE par cible). Une faible MAE indique que le modèle reproduit correctement les cotes marché/FOMO.


In [ ]:
regression_artifacts = {}
regression_metrics = {}
for tf in timeframes:
    artifacts = train_odds_regressors(
        regression_data[tf]["dataset"],
        regression_data[tf]["features"],
        regression_data[tf]["targets"],
    )
    regression_artifacts[tf] = artifacts
    regression_metrics[tf] = artifacts.metrics

regression_metrics


#### Visualisation des cotes prédites

**Objectif** : comparer les sorties du modèle de régression aux probabilités implicites du marché, en utilisant les contrats sélectionnés précédemment. Chaque contrat est tracé séparément afin d’éviter toute liaison entre deux paris distincts.

**Sorties attendues** : trois graphiques (m15, h1, daily) affichant « Marché » vs « Modèle ML ». Les titres suivent le format « Comparaison des cotes <timeframe>\n12 novembre » et l’axe Y est toujours « Probabilité de clôture UP ». Les segments reprennent à 50 % au début de chaque pari sans trait continu entre deux contrats.


In [ ]:
regression_predictions = {}
for tf in timeframes:
    dataset = regression_data[tf]["dataset"].copy()
    predictions = predict_regressions(regression_artifacts[tf], dataset)
    regression_predictions[tf] = predictions

    original_idx = dataset["original_index"]
    target_alias = {
        f"{tf}_prob_up_market": f"{tf}_pred_prob_up_ml",
        f"{tf}_price_up_mid": f"{tf}_pred_price_up_mid_ml",
        f"{tf}_price_down_mid": f"{tf}_pred_price_down_mid_ml",
    }
    for target, alias in target_alias.items():
        pred_col = f"pred_{target}"
        if pred_col in predictions.columns:
            timeframe_tables[tf].loc[original_idx, alias] = predictions[pred_col].values

    actual_col = f"{tf}_prob_up_market"
    pred_col = f"{tf}_pred_prob_up_ml"
    if pred_col not in timeframe_tables[tf].columns:
        continue

    contracts = [cid for cid in selected_contracts.get(tf, []) if cid]
    label_map = {
        actual_col: "Marché",
        pred_col: "Modèle ML",
    }
    figure = plot_odds_comparison(
        timeframe_tables[tf],
        timeframe_label=tf.upper(),
        predicted_cols=[pred_col],
        actual_col=actual_col,
        contract_col=f"{tf}_contract_id",
        contract_ids=contracts if contracts else None,
        label_map=label_map,
    )
    display(figure)


## Modèle de probabilité réelle (classification)

**Objectif** : estimer la probabilité « réelle » de clôture Up pour chaque contrat, en exploitant l’historique de prix et les features techniques, indépendamment de la cote de marché instantanée.

**Sorties attendues** : un dictionnaire `classification_artifacts` et un tableau `classification_metrics` (AUC, accuracy, Brier). Une AUC > 0.5 indique une meilleure discrimination que le hasard, tandis qu’un Brier faible traduit une bonne calibration.

In [ ]:
classification_artifacts = {}
classification_metrics = {}
for tf in timeframes:
    artifacts = train_outcome_classifiers(
        classification_data[tf]["dataset"],
        classification_data[tf]["features"],
        {tf: classification_data[tf]["target"]},
    )
    classification_artifacts[tf] = artifacts
    classification_metrics[tf] = artifacts.metrics

classification_metrics


#### Diagnostic de calibration

**Objectif** : visualiser la calibration des probabilités prédites par les classifieurs sur l’échantillon de test.

**Sorties attendues** : trois courbes de calibration où la diagonale représente une calibration parfaite. Si les points sont au-dessus (ou en dessous), le modèle sous-estime (ou surestime) la probabilité de clôture Up.


In [ ]:
classification_predictions = {}
for tf in timeframes:
    dataset = classification_data[tf]["dataset"].copy()
    preds = predict_classifications(classification_artifacts[tf], dataset)
    classification_predictions[tf] = preds
    calibration_fig = plot_calibration_curve(
        preds,
        f"pred_proba_{tf}",
        classification_data[tf]["target"],
    )
    display(calibration_fig)


## Cotes synthétiques historiques et backtests

**Objectif** : combiner les probabilités prédites avec les scénarios FOMO pour générer des cotes synthétiques historiques, puis évaluer deux stratégies (2 % capital / 4 % parts) via un backtest.

**Sorties attendues** : dictionnaires `backtest_inputs` et `backtest_results` remplis pour chaque timeframe. Les résultats permettent d’analyser la rentabilité et la robustesse du modèle sur données historiques.

In [ ]:
backtest_inputs = {}
backtest_results = {}

for tf in timeframes:
    minute_df = minute_history[tf].copy().sort_values("timestamp").reset_index(drop=True)

    market_series = (
        timeframe_tables[tf][["timestamp", f"{tf}_prob_up_market"]]
        .dropna()
        .sort_values("timestamp")
    )
    minute_df = pd.merge_asof(
        minute_df,
        market_series,
        on="timestamp",
        direction="backward",
        tolerance=pd.Timedelta(minutes=1),
    )
    minute_df.rename(columns={f"{tf}_prob_up_market": "market_prob"}, inplace=True)
    minute_df["market_prob"] = minute_df["market_prob"].ffill().fillna(0.5)

    cls_dataset, minute_features, _ = build_classification_dataset(minute_df, tf)
    model = classification_artifacts[tf].models[tf]
    probabilities = model.predict_proba(cls_dataset[minute_features])[:, 1]
    indices = cls_dataset["original_index"].values
    minute_df.loc[indices, "prediction"] = probabilities

    price_frame = probabilities_to_prices(
        pd.Series(minute_df.loc[indices, "market_prob"].values),
        average_spreads[tf]["spread_up"],
        average_spreads[tf]["spread_down"],
    ).reset_index(drop=True)

    backtest_frame = pd.DataFrame(
        {
            "timestamp": minute_df.loc[indices, "timestamp"].values,
            "prediction": minute_df.loc[indices, "prediction"].values,
            "market_prob": minute_df.loc[indices, "market_prob"].values,
            "outcome": minute_df.loc[indices, f"{tf}_target_up"].values,
        }
    ).reset_index(drop=True)
    backtest_frame = pd.concat([backtest_frame, price_frame], axis=1)
    backtest_inputs[tf] = backtest_frame

    params = BacktestParams(
        timeframe=tf,
        prediction_col="prediction",
        market_prob_col="market_prob",
        outcome_col="outcome",
        price_up_col="price_up_ask",
        price_down_col="price_down_ask",
        threshold=0.05,
        initial_capital=1_000.0,
        capital_risk_fraction=0.02,
        share_fraction=0.04,
    )
    backtest_results[tf] = run_backtest(params, backtest_frame)

backtest_inputs


#### Synthèse tabulaire des backtests

**Objectif** : consolider, pour chaque horizon, le nombre de trades, le winrate, le PnL total et l’équity finale pour les deux stratégies.

**Sorties attendues** : un dictionnaire `backtest_summaries` où chaque `DataFrame` présente les colonnes `trades`, `winrate`, `total_pnl`, `final_equity`. Utilisez ces valeurs pour repérer les horizons les plus rentables ou instables.


In [ ]:
backtest_summaries = {tf: result.summary for tf, result in backtest_results.items()}
backtest_summaries


In [ ]:
for tf in timeframes:
    equity_fig = plot_equity_curves(backtest_results[tf])
    display(equity_fig)


In [ ]:
exported_regression = {
    tf: export_regression_artifacts(regression_artifacts[tf], prefix=f"odds_{tf}")
    for tf in timeframes
}
exported_classification = {
    tf: export_classification_artifacts(classification_artifacts[tf], prefix=f"outcome_{tf}")
    for tf in timeframes
}

exported_regression, exported_classification


## Synthèse rapide

- Les métriques des modèles de cotes et de probabilité sont disponibles dans `regression_metrics` et `classification_metrics`.
- Les backtests pour chaque horizon sont accessibles via `backtest_summaries` et les courbes d'équité.
- Les modèles exportés (régression & classification) sont sauvegardés dans `models/` avec un préfixe par timeframe, utilisables directement via `code.main.run_live_inference`.
